# BrainF Interprter
#### Sanket Tarafder

BrainF is an [esoteric programming language](https://en.wikipedia.org/wiki/Esoteric_programming_language) by Urban Müller. It is created in 1993. 

Notable for its extreme minimalism, the language consists of only eight simple commands and an instruction pointer. While it is fully Turing complete, it is not intended for practical use, but to challenge and amuse programmers. Brainfuck simply requires one to break commands into microscopic steps.

The language's name is a reference to the slang term brainfuck, which refers to things so complicated or unusual that they exceed the limits of one's understanding.

## History

In 1992, Urban Müller, a Swiss physics student, took over a small online archive for Amiga software. The archive grew more popular, and was soon mirrored around the world. Today, it is the world's largest Amiga archive, known as Aminet.

Müller designed Brainfuck with the goal of implementing the smallest possible compiler, inspired by the 1024-byte compiler for the FALSE programming language. Müller's original compiler was implemented in machine language and compiled to a binary with a size of 296 bytes. He uploaded the first Brainfuck compiler to Aminet in 1993. The program came with a "Readme" file, which briefly described the language, and challenged the reader "Who can program anything useful with it? :)". Müller also included an interpreter and some quite elaborate examples. A second version of the compiler used only 240 bytes.

As Aminet grew, the compiler became popular among the Amiga community, and in time it was implemented for other platforms.

## Language design

The language consists of eight commands, listed below. A brainfuck program is a sequence of these commands, possibly interspersed with other characters (which are ignored). The commands are executed sequentially, with some exceptions: an instruction pointer begins at the first command, and each command it points to is executed, after which it normally moves forward to the next command. The program terminates when the instruction pointer moves past the last command.

The brainfuck language uses a simple machine model consisting of the program and instruction pointer, as well as a one-dimensional array of at least 30,000 byte cells initialized to zero; a movable data pointer (initialized to point to the leftmost byte of the array); and two streams of bytes for input and output (most often connected to a keyboard and a monitor respectively, and using the ASCII character encoding).

## Commands

The **[C](https://en.wikipedia.org/wiki/C_(programming_language))** equivalents of the instructions are

| BrainF Command | C Equivalent|
| :---------:    |  :--------  |
|(Program Start) | char array[30000] = {0}; char *ptr = array;|
|>| ++ptr;|
|<| --ptr;|
|+| ++*ptr;|
|-| --*ptr;|
|.| printf("%c", (char)tape\[i\]);|
|,|*ptr = getchar();|
|\[|while (*ptr) {|
|]|}|

For further knowledge referred to the [Wikipedia](https://en.wikipedia.org/wiki/Brainfuck) page and you can find more Brainf Codes in this url[\(just click here\)](http://copy.sh/brainfuck/).

## My Implementation

I have implemented the BrainF Interpreter with [Python](https://en.wikipedia.org/wiki/Python_(programming_language)).

In [1]:
# Mandatory imports
import numpy as np
import sys
from pathlib import Path

#### Reading the code

In my interpreter I have taken the code input from a **.b** file(.txt is also supported).

In [2]:
def get_code(location):
    '''
    
    
    The function asks user for the path of the BrainF code file and reads the
    code in it and returns it.
    
    Returns
    -------
    String
        The BrainF code read from the file.

    '''
    
    path = Path(location)
    
    if path.exists:
        with open(path, mode="r") as file:
            code = file.read()
    else:
        print("No file exist")
        
    return code.strip()

#### Code Validation

Programmers do lots of mistakes while writing codes and this language is prone to have mismatched paranthesis(\[ or \]). So the next function will check whether the loops are valid or not, mainly it will check if it has closing parantheses for all the opening parantheses or vice versa. If it fails to satisfy the criteria the program will terminate its execution.

In [3]:
def validator(code):
    '''
    

    Parameters
    ----------
    code : String
        The BrainF code read from the file.
        
    The function checks the code if it has valid loops or not and exits the
    execution if it finds any wrong loops.

    Returns
    -------
    None.

    '''
    
    stk = []
    
    for e in code:
        if e =="[":
            stk.append(e)
        elif e == "]":
            try:
                stk.pop()
            except:
                print("The code has extra \"]\"")
                sys.exit()
                
    if len(stk) > 0:
        print("The code has extra \"[\"")
        sys.exit()

#### Pairing loops

The programs contain loops and we know if the condition fails the compiler is used to skip the block. For that kind of behaviour we have to pair the loops that we can skip the block without any mistake. So I have made a function to make a dictionary pairing the loop start-end indices.

In [4]:
def get_pairs(code):
    '''
    

    Parameters
    ----------
    code : String
        The BrainF code read from the file.
        
    The function determines the starting and ending position of the loops and
    make pairs of them.

    Returns
    -------
    pairs : Dictionary
        A dictionary having the loop start index as key and ending as value.

    '''
    
    pairs = dict()
    
    position = 0
    index = 0
    
    startb = 0
    endb = 0
    
    while index < len(code):
        if code[index] == "[":
            start, position = index, index + 1
        
        count = 0
        
        while position < len(code):
            if code[position] == "[":
                    count += 1
                    startb += 1
                    
            elif code[position] == "]":
                if count % 2 == 0:
                    if startb == endb:
                        pairs[start] = position
                        break
                    else:
                        endb += 1
                else:
                    count += 1
                    endb += 1
                    
            position += 1
            
        index += 1
        
    return pairs

#### Interpreting the code

All the preprocessings are completed. Now it is the time to interprete the code. I have made a function named **compilR()**, it will interprete the code.

In [5]:
def compilR(code, pairs):
    '''
    

    Parameters
    ----------
    code : String
        The BrainF code read from the file.
    pairs : Dictionary
        A dictionary having the loop start index as key and ending as value.
        
    Compiles the BrainF code.

    Returns
    -------
    None.

    '''
    
    tape = np.zeros(30000, dtype=np.uint8) # Allocating the memory strip for all the operations
    pos = [] # List behaving as a stack to store the loops starting indices
    index = 0 # Index in the tape array
    position = 0 # Position of the pointer in the code
    
    while(position < len(code)):
        e = code[position]
        
        if e == "+":
            tape[index] += 1
        elif e == "-":
            tape[index] -= 1
        elif e == ">":
            index += 1
        elif e == "<":
            index -= 1
        elif e == ".":
            print(chr(tape[index]),end="")
        elif e == ",":
            tape[index] = np.uint8(ord(input()))            
        elif e == "[":            
            if tape[index] != 0:
                pos.append(position)
            else:
                position = pairs[position]                
        elif e == "]":
            position = pos.pop()
            continue
            
        position += 1

#### Driver Function

Now we have to drive the whole code with a driver function which will call the above function as per requirement.

In [6]:
def driver(location):
    code = get_code(location)
    validator(code)
    pairs = get_pairs(code)
    compilR(code, pairs)
    

Run the driver() function with the file location as a argument to recieve the BrainF code from the file.

### Samples

#### Hello World

In [7]:
driver("Samples/hellom.b")

Hello World!


#### PI Calculation Program

In [8]:
driver("Samples/piCal.b")

3.14070455282885


#### 99 bottles in 1752 BrainF instructions by Jim Crawford (http://www.goombas.org/)

In [9]:
driver("Samples/beers.b")

99 Bottles of beer on the wall
99 Bottles of beer
Take one down and pass it around
98 Bottles of beer on the wall

98 Bottles of beer on the wall
98 Bottles of beer
Take one down and pass it around
97 Bottles of beer on the wall

97 Bottles of beer on the wall
97 Bottles of beer
Take one down and pass it around
96 Bottles of beer on the wall

96 Bottles of beer on the wall
96 Bottles of beer
Take one down and pass it around
95 Bottles of beer on the wall

95 Bottles of beer on the wall
95 Bottles of beer
Take one down and pass it around
94 Bottles of beer on the wall

94 Bottles of beer on the wall
94 Bottles of beer
Take one down and pass it around
93 Bottles of beer on the wall

93 Bottles of beer on the wall
93 Bottles of beer
Take one down and pass it around
92 Bottles of beer on the wall

92 Bottles of beer on the wall
92 Bottles of beer
Take one down and pass it around
91 Bottles of beer on the wall

91 Bottles of beer on the wall
91 Bottles of beer
Take one down and pass it arou

#### Printing all the Charecters

In [10]:
driver("Samples/chars.b")

 	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ ¡¢£¤¥¦§¨©ª«¬­®¯°±²³´µ¶·¸¹º»¼½¾¿ÀÁÂÃÄÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖ×ØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿ

#### Draws shapes

In [11]:
driver("Samples/oobrain.b")

DRAWING A CIRCLE AT:(15,25), RADIUS 8
DRAWING A RECTANGLE AT:(10,20), WIDTH 5, HEIGHT 6
DRAWING A CIRCLE AT:(115,125), RADIUS 8
DRAWING A RECTANGLE AT:(110,120), WIDTH 5, HEIGHT 6
DRAWING A RECTANGLE AT:(0,0), WIDTH 30, HEIGHT 15


#### Outputs square numbers from 0 to 10000

In [12]:
driver("Samples/squares.b")

0
1
4
9
16
25
36
49
64
81
100
121
144
169
196
225
256
289
324
361
400
441
484
529
576
625
676
729
784
841
900
961
1024
1089
1156
1225
1296
1369
1444
1521
1600
1681
1764
1849
1936
2025
2116
2209
2304
2401
2500
2601
2704
2809
2916
3025
3136
3249
3364
3481
3600
3721
3844
3969
4096
4225
4356
4489
4624
4761
4900
5041
5184
5329
5476
5625
5776
5929
6084
6241
6400
6561
6724
6889
7056
7225
7396
7569
7744
7921
8100
8281
8464
8649
8836
9025
9216
9409
9604
9801
10000


#### Printing Sierpinski triangle

In [13]:
driver("Samples/triangle.b")

                                *    
                               * *    
                              *   *    
                             * * * *    
                            *       *    
                           * *     * *    
                          *   *   *   *    
                         * * * * * * * *    
                        *               *    
                       * *             * *    
                      *   *           *   *    
                     * * * *         * * * *    
                    *       *       *       *    
                   * *     * *     * *     * *    
                  *   *   *   *   *   *   *   *    
                 * * * * * * * * * * * * * * * *    
                *                               *    
               * *                             * *    
              *   *                           *   *    
             * * * *                         * * * *    
            *       *                       *     

####  A mandelbrot set fractal viewer by Erik Bosman

My Interpreter is not fast and it can take approximately 2 hours to interprete the code so I have pre interpreted the code in [Google Colab](https://colab.research.google.com/drive/1aDXdQA6YSggwIBd_OHarxQU8k9Njr6BO?usp=sharing).